**Set environment**

In [1]:
### basic
import sys
sys.path.append('../')
from config_sing import *

### specific tools
from functools import reduce
import itertools as it
import sqlite3
# https://stackoverflow.com/questions/49456158/integer-in-python-pandas-becomes-blob-binary-in-sqlite
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

### file path of fragment database
fdiry = os.path.join(FD_RES, "KS91_K562_ASTARRseq", 'database')
fname = "fragment_astarr_gata1.db"
FPATH_DB_ASTARR = os.path.join(fdiry, fname)

fdiry = os.path.join(FD_RES, "Tewhey_K562_TileMPRA", 'database')
fname = "fragment_tilempra_gata1.db"
FPATH_DB_TMPRA = os.path.join(fdiry, fname)

FPATHS_DB = [FPATH_DB_ASTARR, FPATH_DB_TMPRA]

You are on Duke Server: Singularity: Proj ENCODE FCC
BASE DIRECTORY:     /mount/work
PATH OF SOURCE:     /mount/work/source
PATH OF EXECUTABLE: /mount/work/exe
PATH OF ANNOTATION: /mount/work/annotation
PATH OF PROJECT:    /mount/project
PATH OF RESULTS:    /mount/work/out/proj_encode_fcc



## Check: Table & Index

In [2]:
query = """
    SELECT name 
    FROM sqlite_master 
    WHERE type = 'table';
    """

for fpath_db in FPATHS_DB:
    with sqlite3.connect(fpath_db) as conn:
        ### show created table info
        cursor = conn.cursor()
        cursor = cursor.execute(query)
        rows   = cursor.fetchall()
        rows   = np.r_[rows].ravel()
    print(fpath_db)
    for row in np.sort(rows):
        print(f"\t{row}")
    print()

/mount/work/out/proj_encode_fcc/KS91_K562_ASTARRseq/database/fragment_astarr_gata1.db
	Annotation
	Count
	Coverage
	Fragment
	Motif
	Sample

/mount/work/out/proj_encode_fcc/Tewhey_K562_TileMPRA/database/fragment_tilempra_gata1.db
	Annotation
	Count
	Fragment
	Motif



In [3]:
query = """
    SELECT name 
    FROM sqlite_master 
    WHERE type = 'index';
    """

for fpath_db in FPATHS_DB:
    with sqlite3.connect(fpath_db) as conn:
        ### show created table info
        cursor = conn.cursor()
        cursor = cursor.execute(query)
        rows   = cursor.fetchall()
        rows   = np.r_[rows].ravel()
    print(fpath_db)
    for row in np.sort(rows):
        print(f"\t{row}")
    print()

/mount/work/out/proj_encode_fcc/KS91_K562_ASTARRseq/database/fragment_astarr_gata1.db
	idx_annot_frag
	idx_count_sample
	idx_cov_loc
	idx_frag_loc
	idx_motif_loc
	sqlite_autoindex_Annotation_1
	sqlite_autoindex_Fragment_1
	sqlite_autoindex_Motif_1
	sqlite_autoindex_Sample_1

/mount/work/out/proj_encode_fcc/Tewhey_K562_TileMPRA/database/fragment_tilempra_gata1.db
	idx_annot_frag
	idx_frag_loc
	idx_motif_loc
	sqlite_autoindex_Annotation_1
	sqlite_autoindex_Count_1
	sqlite_autoindex_Fragment_1
	sqlite_autoindex_Motif_1



## Check: Sample

In [4]:
fpath_db = FPATH_DB_ASTARR
table    = "Sample"

with sqlite3.connect(fpath_db) as conn:
    ### show created table info
    cursor = conn.cursor()
    query = f"select count(*) from {table}"
    cursor.execute(query)
    print("#Rows Table:", cursor.fetchall())
    
    ### show that the table is created
    cursor = conn.cursor()
    query  = f"SELECT * FROM {table}"
    cursor = cursor.execute(query)
    print(f"Show the first ten rows of table {table}:")
    rows   = it.islice(cursor, 10)
    for row in rows:
        print(row)

#Rows Table: [(10,)]
Show the first ten rows of table Sample:
('Input_rep1', 'Input', 'rep1', 358823)
('Input_rep2', 'Input', 'rep2', 461577)
('Input_rep3', 'Input', 'rep3', 496229)
('Input_rep4', 'Input', 'rep4', 464845)
('Input_rep5', 'Input', 'rep5', 454013)
('Input_rep6', 'Input', 'rep6', 409058)
('Output_rep1', 'Output', 'rep1', 505724)
('Output_rep2', 'Output', 'rep2', 686033)
('Output_rep3', 'Output', 'rep3', 441621)
('Output_rep4', 'Output', 'rep4', 739000)


## Check: the two motif tables should be the same

In [5]:
table = "Motif"

for fpath_db in FPATHS_DB:
    print(fpath_db)
    with sqlite3.connect(fpath_db) as conn:
        ### show created table info
        cursor = conn.cursor()
        query  = f"select count(*) from {table}"
        cursor = cursor.execute(query)
        print("#Rows Table:", cursor.fetchall())

        ### show that the table is created
        cursor = conn.cursor()
        query  = f"SELECT * FROM {table}"
        cursor = cursor.execute(query)
        print(f"Show the first five rows of table {table}:")
        rows   = it.islice(cursor, 5)
        for row in rows:
            print(row)
    print()

/mount/work/out/proj_encode_fcc/KS91_K562_ASTARRseq/database/fragment_astarr_gata1.db
#Rows Table: [(24863390,)]
Show the first five rows of table Motif:
('chrX_10006_10041_KLF/SP/2', 'chrX', 10006, 10041, 'KLF/SP/2', 3.257775)
('chrX_10018_10038_GC-tract', 'chrX', 10018, 10038, 'GC-tract', 6.314)
('chrX_10025_10039_NR/3', 'chrX', 10025, 10039, 'NR/3', 7.0957)
('chrX_10035_10046_PRDM1', 'chrX', 10035, 10046, 'PRDM1', 8.1562)
('chrX_10046_10061_MAF', 'chrX', 10046, 10061, 'MAF', 7.1564)

/mount/work/out/proj_encode_fcc/Tewhey_K562_TileMPRA/database/fragment_tilempra_gata1.db
#Rows Table: [(24863390,)]
Show the first five rows of table Motif:
('chrX_10006_10041_KLF/SP/2', 'chrX', 10006, 10041, 'KLF/SP/2', 3.257775)
('chrX_10018_10038_GC-tract', 'chrX', 10018, 10038, 'GC-tract', 6.314)
('chrX_10025_10039_NR/3', 'chrX', 10025, 10039, 'NR/3', 7.0957)
('chrX_10035_10046_PRDM1', 'chrX', 10035, 10046, 'PRDM1', 8.1562)
('chrX_10046_10061_MAF', 'chrX', 10046, 10061, 'MAF', 7.1564)



## Check: Count

In [6]:
table = "Count"

for fpath_db in FPATHS_DB:
    print(fpath_db)
    with sqlite3.connect(fpath_db) as conn:
        ### show created table info
        cursor = conn.cursor()
        query  = f"select count(*) from {table}"
        cursor = cursor.execute(query)
        print("#Rows Table:", cursor.fetchall())

        ### show that the table is created
        cursor = conn.cursor()
        query  = f"SELECT * FROM {table}"
        cursor = cursor.execute(query)
        print(f"Show the first five rows of table {table}:")
        rows   = it.islice(cursor, 5)
        for row in rows:
            print(row)
    print()

/mount/work/out/proj_encode_fcc/KS91_K562_ASTARRseq/database/fragment_astarr_gata1.db
#Rows Table: [(2557129,)]
Show the first five rows of table Count:
('chrX_47787165_47787363', 'Input_rep1', 1)
('chrX_47787557_47787773', 'Input_rep1', 1)
('chrX_47787593_47787783', 'Input_rep1', 1)
('chrX_47787598_47787772', 'Input_rep1', 1)
('chrX_47787661_47788058', 'Input_rep1', 1)

/mount/work/out/proj_encode_fcc/Tewhey_K562_TileMPRA/database/fragment_tilempra_gata1.db
#Rows Table: [(52312,)]
Show the first five rows of table Count:
('chrX_47786401_47786600', 2.30593934460149, 2117.78684083014, 10474.868702612, 112.11265261617, 110.931130790033, 0.102147903305184)
('chrX_47786451_47786650', 0.783056057422144, 1386.00739237509, 2387.11896574661, 11.3173883956403, 10.6425168461249, 0.113307047070907)
('chrX_47786501_47786700', -0.128731962724311, 1955.62553108109, 1788.23614005802, 0.845069660219201, 0.628722618349189, 0.0878596785422427)
('chrX_47786551_47786750', 0.588295148181817, 2388.996642614

## Check: Fragment

In [7]:
table = "Fragment"

for fpath_db in FPATHS_DB:
    print(fpath_db)
    with sqlite3.connect(fpath_db) as conn:
        ### show created table info
        cursor = conn.cursor()
        query  = f"select count(*) from {table}"
        cursor = cursor.execute(query)
        print("#Rows Table:", cursor.fetchall())

        ### show that the table is created
        cursor = conn.cursor()
        query  = f"SELECT * FROM {table}"
        cursor = cursor.execute(query)
        print(f"Show the first five rows of table {table}:")
        rows   = it.islice(cursor, 5)
        for row in rows:
            print(row)
    print()

/mount/work/out/proj_encode_fcc/KS91_K562_ASTARRseq/database/fragment_astarr_gata1.db
#Rows Table: [(1220319,)]
Show the first five rows of table Fragment:
('chrX_47787165_47787363', 'chrX', 47787165, 47787363, 0.560606, 0.439394, 33, 34, 53, 78, 0, 0)
('chrX_47787557_47787773', 'chrX', 47787557, 47787773, 0.615741, 0.384259, 45, 35, 48, 88, 0, 0)
('chrX_47787593_47787783', 'chrX', 47787593, 47787783, 0.636842, 0.363158, 42, 32, 37, 79, 0, 0)
('chrX_47787598_47787772', 'chrX', 47787598, 47787772, 0.637931, 0.362069, 37, 28, 35, 74, 0, 0)
('chrX_47787661_47788058', 'chrX', 47787661, 47788058, 0.594458, 0.405542, 76, 72, 89, 160, 0, 0)

/mount/work/out/proj_encode_fcc/Tewhey_K562_TileMPRA/database/fragment_tilempra_gata1.db
#Rows Table: [(52312,)]
Show the first five rows of table Fragment:
('chrX_47786401_47786600', 'chrX', 47786401, 47786600, 0.668342, 0.331658, 33, 27, 39, 100, 0, 0)
('chrX_47786451_47786650', 'chrX', 47786451, 47786650, 0.658291, 0.341709, 36, 30, 38, 95, 0, 0)
('chr

## Check: Annotation

In [8]:
table = "Annotation"

for fpath_db in FPATHS_DB:
    print(fpath_db)
    with sqlite3.connect(fpath_db) as conn:
        ### show created table info
        cursor = conn.cursor()
        query  = f"select count(*) from {table}"
        cursor = cursor.execute(query)
        print("#Rows Table:", cursor.fetchall())

        ### show that the table is created
        cursor = conn.cursor()
        query  = f"SELECT * FROM {table}"
        cursor = cursor.execute(query)
        print(f"Show the first five rows of table {table}:")
        rows   = it.islice(cursor, 5)
        for row in rows:
            print(row)
    print()

/mount/work/out/proj_encode_fcc/KS91_K562_ASTARRseq/database/fragment_astarr_gata1.db
#Rows Table: [(85201024,)]
Show the first five rows of table Annotation:
('chrX_47787165_47787363', 'chrX_47787180_47787190_FOX/8')
('chrX_47787165_47787363', 'chrX_47787182_47787192_HINFP1/1')
('chrX_47787165_47787363', 'chrX_47787194_47787216_ZNF146')
('chrX_47787165_47787363', 'chrX_47787196_47787211_NR/15')
('chrX_47787165_47787363', 'chrX_47787229_47787243_NR/20')

/mount/work/out/proj_encode_fcc/Tewhey_K562_TileMPRA/database/fragment_tilempra_gata1.db
#Rows Table: [(2300625,)]
Show the first five rows of table Annotation:
('chrX_47786401_47786600', 'chrX_47786401_47786418_KLF/SP/2')
('chrX_47786401_47786600', 'chrX_47786424_47786443_ZNF28')
('chrX_47786401_47786600', 'chrX_47786449_47786459_FOX/4')
('chrX_47786401_47786600', 'chrX_47786460_47786481_ZNF382')
('chrX_47786401_47786600', 'chrX_47786461_47786483_ZNF136')



## Check: Coverage

In [9]:
fpath_db = FPATH_DB_ASTARR
table    = "Coverage"

with sqlite3.connect(fpath_db) as conn:
    ### show created table info
    cursor = conn.cursor()
    query = f"select count(*) from {table}"
    cursor.execute(query)
    print("#Rows Table:", cursor.fetchall())
    
    ### show that the table is created
    cursor = conn.cursor()
    query  = f"SELECT * FROM {table}"
    cursor = cursor.execute(query)
    print(f"Show the first ten rows of table {table}:")
    rows   = it.islice(cursor, 10)
    for row in rows:
        print(row)

#Rows Table: [(18003141,)]
Show the first ten rows of table Coverage:
('chrX', 47786401, 0, 'Input_rep1')
('chrX', 47786402, 0, 'Input_rep1')
('chrX', 47786403, 0, 'Input_rep1')
('chrX', 47786404, 0, 'Input_rep1')
('chrX', 47786405, 0, 'Input_rep1')
('chrX', 47786406, 0, 'Input_rep1')
('chrX', 47786407, 0, 'Input_rep1')
('chrX', 47786408, 0, 'Input_rep1')
('chrX', 47786409, 0, 'Input_rep1')
('chrX', 47786410, 0, 'Input_rep1')
